# Imports

In [ ]:
import glob
from PIL import ImageDraw
from PIL import Image
import numpy as np
import re
import codecs
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import ntpath 
import tensorflow as tf
import copy

## Hyperparameter

In [ ]:
SCALE_PATH = 'all_scale_data.npy'

OUTSIZE = (200, 100)
SCALE_WIDTH_CM = 2  # Breite in cm der Verbindungsgerade
                    # zwischen den Skalenendpunkten
SCALE_PADDING_FACTOR = 1.4 # Faktor der angibt um wie viel breiter
                           # die Skala ist, als die Verbindungs-
                           # gerade zwischen den Skalenpunkten.
        
                           # Im gebenenen Fall befinden sich die 
                           # Skalenendpunkte 2cm von einander entfernt.
                           # Alle Skalen sind aber links und rechts um 
                           # mindestens 0.4cm breiter. Demnach haben sie
                           # eine Gesamtbreite von 2.8cm = 1.4 * 2cm.
NUM_TRANSLATIONS = 5 # Anzahl der Zufallstranslationen pro Bild
TRANSLATION_DELTA = 30   # Translationen werden aus dem Bereich 
                        # [-TRANSLATION_DELTA, TRANSLATION_DELTA]
                        # gewuerfelt

INFOLDER = 'DIP_images_fresh/all/'
OUTFOLDER = f"out_all/smallbox2/"

In [ ]:
# n = len(inpaths)
# for i in range(n):    
#     inpath = inpaths[i]
#     filename = inpath[len(infolder):]
#     img = Image.open(inpath)
#     landmarks = scale_data[filename]['landmarks']
#     img = draw_landmarks(img, landmarks, 3)
#     img.save(f"out_all/2/{filename}")

## Funktionen für Rechnung in der Affinenen Ebene

In [ ]:
#Bogenmaß in Grad umrechnen
def rad_to_deg(rad):
    return rad * 360 / (2*np.pi) 

#Grad in Bogenmaß umrechnen
def deg_to_rad(deg):
    return deg / 360 * (2*np.pi)

Wende die Affine Rotation
$\mathbb{A}_{\varphi}(v) = O_{\varphi}v + m$
mit Rotationsmatrix $O_{\varphi} = \begin{pmatrix} \operatorname{cos}(\varphi) & \operatorname{sin}(\varphi) \\ - \operatorname{sin}(\varphi) & \operatorname{cos}(\varphi) \end{pmatrix}$ um Mittelpunkt $m$ auf mehrere Vektoren $v = \text{points}$ an.

In [ ]:
# Berechnet Koordinaten von points
# in affinen Ebene nach rotieren um den center
# Punkt um einen Winkel angle
def affine_rotation(angle, middle, points):
    
    phi = deg_to_rad(angle) #Grad in Bogenmaß
    
    O = np.array([[np.cos(phi), np.sin(phi)],[-np.sin(phi), np.cos(phi)]]).reshape([2,2]) #Drehmatrix
    v = points - middle
    
    flat = len(v.shape) == 1
    
    if flat:
        v = v.reshape([1,2])
        
    #neue Koordinaten ermitteln durch Multiplikation mit Drehmatrix und Addition der Mitte
    result = np.einsum('ij,kj -> ki', O, v) + middle      
    
    if flat:
        result = result.flatten()
    
    return result


Wende die inverse Affine Rotation
$\mathbb{A}_{\varphi}(v) = O^\intercal_{\varphi}v + m$
mit Rotationsmatrix $O^\intercal_{\varphi} = \begin{pmatrix} \operatorname{cos}(\varphi) & -\operatorname{sin}(\varphi) \\ \operatorname{sin}(\varphi) & \operatorname{cos}(\varphi) \end{pmatrix}$ um Mittelpunkt $m$ auf mehrere Vektoren $v = \text{points}$ an.

In [ ]:
def inverse_affine_rotation(angle, middle, points):
    
    phi = deg_to_rad(angle) #Grad in Bogenmaß
    
    O = np.array([[np.cos(phi), np.sin(phi)],[-np.sin(phi), np.cos(phi)]]).reshape([2,2]) #Drehmatrix
    v = points - middle
    
    flat = len(v.shape) == 1
    
    if flat:
        v = v.reshape([1,2])
        
    #neue Koordinaten ermitteln durch Multiplikation mit Drehmatrix und Addition der Mitte
    result = np.einsum('ij,kj -> ki', O.T, v) + middle     
    
    if flat:
        result = result.flatten()
    
    return result

Berechne den Geometrischen Schwerpunkt mittels s = $\frac{1}{m}\cdot\sum\limits_{i=1}^{m} x_{i} $

In [ ]:
# Geometrischer Schwerpunkt mehrerer Punkte im Raum
def centroid(points):
    return np.mean(points, axis=0)

In [ ]:
# verschiebung des Nullpunkts auf den Punkt new_origin
def affine_translation(new_origin, points):
    return points - new_origin

# Verarbeitung

## Rotationsaffinität für gegebenen Skalendaten eines Bilds berechnen

In [ ]:
def affine_rotation_from_scale_data(img_dimension, scale_data, top_offset = 300):
    
    left = scale_data['left']
    right = scale_data['right']
    x1 = left[0]
    y1 = left[1]
    x2 = right[0]
    y2 = right[1]
    
    l = np.array([x1,y1])
    r = np.array([x2,y2])
    angle = 0

    if (y2 != y1):

        alpha = np.arctan(np.abs(y2-y1) / np.abs(x2-x1)) #Winkel ausrechnen
        angle = rad_to_deg(alpha)
        angle = -np.sign(y2-y1) * angle #Vorzeichen ermitteln, um richtig zu drehen

    middle = np.array( [img_dimension[0] / 2, img_dimension[1] / 2] ) #rotate dreht um den Mittelpunkt

    return angle, middle

## Rotationsaffinität von Skalendaten benutzen um eine passende Box auszuschneiden

In [ ]:
def crop_from_scale_affinity(img, angle, middle, scale_data, top_offset = 300, scale_padding_factor=SCALE_PADDING_FACTOR):
    
    l = np.array(scale_data['left'])
    r = np.array(scale_data['right'])
    
    l[1] = img.height-l[1] #auf andere Skala umrechnen mit (0,0) oben links
    r[1] = img.height-r[1] 
    
    scale_data = np.array([ [l[0], l[1]], [r[0], r[1]] ])
    scale_data_rotated = affine_rotation(angle, middle, scale_data)
    lnew = scale_data_rotated[0,:]
    rnew = scale_data_rotated[1,:]
    
    img = img.rotate(angle) #rotieren mit Winkel als Grad

    v = rnew - lnew
    lnew = lnew - (scale_padding_factor - 1)/2*v
    rnew = rnew + (scale_padding_factor - 1)/2*v
    w = np.array([ v[1], -v[0] ])
    ulnew = lnew + w
    ulnew[1] = ulnew[1] - top_offset

    box = (ulnew[0],ulnew[1],rnew[0],rnew[1]) #Box definieren

    cropped = img.crop(box) #Bild zuschneiden
    
    return cropped, box

## Affine Rotation so bestimmen, dass "möglichst viel grün gerade ist"

In [ ]:
def affine_rotation_from_scale_data_crop(cropped):
    im_rgb = cropped.convert("RGB")
    I = np.transpose(np.array(im_rgb), (1,0,2))
    I = I.astype(np.int64)

    # schwellwerte fuer die einzelnen farbkanaele 
    # Dabei verbleiben: der Strang, weisser Hintergrund,
    #                    gruene Klammern
    wlim = 10
    klim = 100

    # Bestimme die einzelnen Farbkanaele
    R = I[:,:,0]
    G = I[:,:,1]
    B = I[:,:,2]

    # fuer die Erkennung des weissen Hintergrunds
    W = np.max( np.c_[ np.abs(R-G)[:,:,np.newaxis], np.abs(B-G)[:,:,np.newaxis], np.abs(R-B)[:,:,np.newaxis] ], axis=2)

    # Berechne die Bildmaske zur Erkennung des Strangs
    J = (R <= G) & (B <= G) & (W > wlim) 
    x,y = J.nonzero()
    reg = LinearRegression(fit_intercept = True).fit(x[:,np.newaxis],y)
    
    reg_coef = reg.coef_[0]
    
    line = reg.intercept_+reg.coef_*x
    plt.imshow(J.T, cmap = 'gray')
    plt.plot(x,line)
    
    angle = np.arctan(1 / np.abs(reg_coef)) * 180 / np.pi
    angle = np.sign(reg_coef) * (90-angle)
    middle = np.array( [cropped.width / 2, cropped.height / 2] ) #rotate dreht um den Mittelpunkt

    return angle, middle

In [ ]:
#affine_rotation_from_scale_data_crop(Image.open("out_crop/DIPalp16_2_d_3.jpg"))

### Für Testzwecke

In [ ]:
def draw_landmarks(img, landmarks, radius = 10):
    lm = landmarks
    draw = ImageDraw.Draw(img)
    for i in range(lm.shape[0]):
        draw.ellipse((lm[i,0]-radius,lm[i,1]-radius, lm[i,0]+radius,lm[i,1]+radius),fill = 'red')
    return img

In [ ]:
scale_data = np.load(SCALE_PATH, allow_pickle = True)
scale_data = scale_data[()]
inpaths = glob.glob(INFOLDER + '*.jpg')

outratio = OUTSIZE[0]/OUTSIZE[1]

n = len(inpaths)   
# Zufalls Translationen fuer x- und y-Koordinaten
translations = np.random.randint(low=-TRANSLATION_DELTA,high=TRANSLATION_DELTA, size = (n,NUM_TRANSLATIONS,2))
print(len(translations))
transformed_landmarks = {}

debug_steps = 10

for i in range(n):
    if i%debug_steps == 0:
        print(f"{i+1} / {n}")
            
    inpath = inpaths[i]
    filename = inpath[len(INFOLDER):]
    img = Image.open(inpath)
    
    landmarks = copy.deepcopy(scale_data[filename]['landmarks'])
   
    a1, m1 = affine_rotation_from_scale_data(img.size, scale_data[filename])
    cropped, box = crop_from_scale_affinity(img, a1, m1, scale_data[filename])
    a2, m2_local = affine_rotation_from_scale_data_crop(cropped)
    
    #Komposition beider Winkel
    angle = a1 + a2
    #Verhältnis von Pixel zu cm 
    # Skalenendpunkte 2 cm voneinander entfernt, 
    #in crop_from_scale_affinity 1.4 fache drauf --> 2.8cm
    box_width_in_cm = SCALE_PADDING_FACTOR * SCALE_WIDTH_CM
    cm_pixel_ratio = (box[2] - box[0]) / box_width_in_cm #x/2.8 = 1cm
    
    #Mitte der Landmarken und des Bildes ausrechnen
    middle_landmarks = centroid(landmarks)
    middle_image = np.array(img.size) / 2
    
    #Verschiebungsvektor
    v = middle_landmarks - middle_image
    
    #Mittelpunkt der Landmarken in die Mitte des Bilder verschieben
    img = img.transform(img.size, Image.AFFINE, (1, 0, v[0], 0, 1, v[1]))
    img = img.rotate(angle)

    #neue Landmarkenkoordinaten nach Verschiebung
    landmarks = affine_translation(v, landmarks)
    
    #neue Landmarkenkoordinaten nach der Drehung
    landmarks = affine_rotation(angle, middle_image, landmarks)

    middle_landmarks = centroid(landmarks)
    
    cm_height = SCALE_WIDTH_CM * (1/outratio)
    
    w_left = np.array([(cm_pixel_ratio * cm_width/2), (cm_pixel_ratio*cm_height/2)])
    w_right = np.array([(cm_pixel_ratio * cm_width/2), (cm_pixel_ratio*cm_height/2)])
    
    left = middle_image - w_left 
    right = middle_image + w_right
    
    T = np.array(np.r_[np.matrix(((0,0))), translations[i,...]])
    
    
    for t in range(NUM_TRANSLATIONS+1):
        translation = T[t,:]
        
        left_t = left + translation
        right_t = right + translation

        box = (left_t[0],left_t[1],right_t[0],right_t[1]) 
        
        cropped = img.crop(box)

        #neue Landmark koordinaten nach dem Zuschneiden
        landmarks_t = affine_translation(left_t, landmarks)
        cropped = cropped.resize(OUTSIZE)
        
        #neue Landmarkenkoordinaten nach dem Skalieren
        scaling_factor = (right_t[0]-left_t[0])/outsize[0]
        landmarks_t = landmarks_t / scaling_factor
       
        #draw_landmarks(cropped, landmarks_t, radius = 2)
        
        filename_t = filename[:-len(".jpg")] + f"_t{t}.jpg"
        cropped.save(OUTFOLDER + filename_t)
        
        transformed_landmarks[filename_t] = {
            'landmarks' : landmarks_t
        }

np.save(f'preprocessed_landmarks.npy',transformed_landmarks)

In [ ]:
# # eine kollage erstellen mit allen 
# # landmark bildern
# columns = 10
# lm_radius = 1
# rows = math.ceil(len(scale_data) / columns)
# filenames = list(transformed_landmarks.keys())

# collage = Image.new('RGB', (columns * OUTSIZE[0], rows * OUTSIZE[1]))

# for i in range(len(filenames)):
#     filename = filenames[i]
#     x = (i % columns) * OUTSIZE[0]
#     y = (i // columns) * OUTSIZE[1]
    
#     lm = transformed_landmarks[filename]['landmarks']
#     img = Image.open(OUTFOLDER + filename)
#     draw = ImageDraw.Draw(img)
#     for i in range(lm.shape[0]):
#         draw.ellipse((lm[i,0]-lm_radius,lm[i,1]-lm_radius, lm[i,0]+lm_radius,lm[i,1]+lm_radius),fill = 'red')

#     collage.paste(img, (x, y))
# collage.save(f'all_collage_easy2.jpg')